In [4]:
import email.message
import mimetypes
import os.path
import smtplib
import csv
import time
from datetime import datetime

In [5]:
def generate_message(sender, recipient, subject, body, attachment_path):
    ### Creates an email with an attachement ###
    # Basic Email formatting
    message = email.message.EmailMessage()
    message["From"] = sender
    message["To"] = recipient
    message["Subject"] = subject
    message.set_content(body)

    # Process the attachment and add it to the email
    attachment_filename = os.path.basename(attachment_path)
    mime_type, _ = mimetypes.guess_type(attachment_path)
    mime_type, mime_subtype = mime_type.split('/', 1)

    with open(attachment_path, 'rb') as ap:
        message.add_attachment(ap.read(),
            maintype=mime_type,
            subtype=mime_subtype,
            filename=attachment_filename)

    return message

def send(message):
    try:
        ### Sends the message to the configured SMTP server.###
        mail_server = smtplib.SMTP('smtp.gmail.com', 587)
        mail_server.starttls()
        mail_server.login('coltonboxell@gmail.com', 'ujkciovzopyfbusu')
        mail_server.send_message(message)
        mail_server.quit()
        return 0
        # This is crap, but I'm not sure how to make the error reporting better. 
        # I want to save it to a log file, but I need the intended recipient for proper logging  
    except:
        return 1

In [6]:
attachment_path = "/home/colton/Documents/Resume.pdf"
counter = 1

with open('DailyResults.csv', 'r') as dr:
    for row in csv.DictReader(dr):
        Job_ID, Title, Date, Company, Location, Salary, Job_Source, Email = row.values()
        if Email:
            recipient = Email
            subject = """{} Application""".format(Title)
            body = """Hello {},

            My name is Colton Boxell, I am applying to the position of {} with {} located in {}

            My resume is attached.

            I am contacting you through my automated Python service.

            Thank you and have a wonderful day

             Colton Boxell
            """.format(Company, Title, Company, Location)

            message = generate_message("coltonboxell@gmail.com", recipient, subject, body, attachment_path)
            status = send(message)
            #This needs to be changed to a sent column rather than a separate file
            with open("sent_log", "a") as sl:
                now = datetime.now()
                sl.write("Application for {} sent to {} at {} with result {}\n".format(Title, recipient, now, status))
            print("{} Messages sent in this process!".format(counter), end = "\r", flush=True)
            counter += 1
            time.sleep(4)
        else:
            continue

ValueError: too many values to unpack (expected 8)